In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest

import eli5
from eli5.sklearn import PermutationImportance


In [2]:
def exclude_correlated_features_trainingset(input_df, threshold =0.90):
    df = input_df.copy()
    
    cor_matrix = df.corr().abs()
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    
    new_df = df.drop(df[to_drop], axis=1)
    
    return new_df, to_drop

In [3]:
def drop_correlated_features_evaluation(input_df, to_drop):
    df = input_df.copy()
    new_df = df.drop(df[to_drop], axis=1)
    return new_df

## Load the dataset.

In [4]:
df = pd.read_csv('hcc_mortality_risk_prediction.csv')
df

,log-sigma-1-0-mm-3D_firstorder_10Percentile_pre,log-sigma-1-0-mm-3D_firstorder_90Percentile_pre,log-sigma-1-0-mm-3D_firstorder_Energy_pre,log-sigma-1-0-mm-3D_firstorder_Entropy_pre,log-sigma-1-0-mm-3D_firstorder_InterquartileRange_pre,log-sigma-1-0-mm-3D_firstorder_Kurtosis_pre,log-sigma-1-0-mm-3D_firstorder_Maximum_pre,log-sigma-1-0-mm-3D_firstorder_MeanAbsoluteDeviation_pre,log-sigma-1-0-mm-3D_firstorder_Mean_pre,log-sigma-1-0-mm-3D_firstorder_Median_pre,...,lab_total_bili,lab_serum_alb,lab_INR,lab_sodium,lab_creatinine,lab_direct_bili,lab_ptt,METS,DEAD,SURVIVAL
0,-10.059183,5.745775,4.107325e+07,1.079902,7.013954,9.529045,82.802605,5.188680,-1.943173,-1.266524,...,1.33,2.9,1.07,138.0,1.100,0.430,11.70,1.0,True,105
1,-9.126845,5.881439,6.134392e+07,1.083674,6.751441,8.286864,82.595139,4.918833,-1.306885,-0.704091,...,0.30,3.5,0.97,141.0,1.170,0.200,10.90,0.0,False,974
2,-15.583829,10.506743,6.463516e+07,1.264821,9.072473,7.380540,100.180374,7.854720,-2.162958,-0.544937,...,1.19,3.0,1.60,139.0,0.775,0.505,15.85,0.0,True,212
3,-13.333085,11.715394,1.642372e+08,1.178096,12.550628,6.660825,121.306679,7.978263,-0.824225,-0.844899,...,0.70,3.7,0.90,136.0,0.730,0.215,11.30,0.0,False,856
4,-19.478720,15.217196,1.075018e+08,1.429544,17.504348,4.219180,87.393265,11.002891,-1.919913,-1.638360,...,3.46,3.1,1.16,131.0,0.800,1.130,13.30,0.0,True,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,-9.472473,6.006749,3.779490e+07,1.046511,7.333974,6.446112,64.962936,4.996937,-1.438888,-1.064579,...,1.48,2.7,1.17,135.0,1.000,0.640,13.00,0.0,True,500
551,-11.148445,6.849886,6.125577e+07,1.142745,8.217744,8.180785,90.151703,5.956414,-1.833336,-1.201947,...,1.00,4.0,1.04,141.0,0.760,0.300,11.50,0.0,False,858
552,-14.544942,10.197468,6.777509e+07,1.231157,11.572974,5.518432,75.665199,7.913425,-1.933288,-1.501237,...,1.50,3.5,1.11,139.0,0.700,1.230,14.10,0.0,False,494
553,-9.644036,6.424202,2.951602e+07,1.029232,7.437437,5.661589,51.036461,5.060680,-1.447211,-1.028005,...,0.82,3.2,0.98,138.0,0.500,0.420,11.70,0.0,True,760


## Train the framework.

In [5]:
df_train, df_test = train_test_split(df,test_size=0.15, random_state=111)
 
X_train = df_train.drop(["DEAD","SURVIVAL"], axis=1)
X_test = df_test.drop(["DEAD","SURVIVAL"], axis=1)    
    
y_train = df_train[["DEAD","SURVIVAL"]].to_records(index=False)    
y_test = df_test[["DEAD","SURVIVAL"]].to_records(index=False)

## Exclusion of all candidate variables with a Pearson correlation coefficient >0.9 in the development cohort to reduce multicollinearity.

In [6]:
X_train, to_drop = exclude_correlated_features_trainingset(X_train)
print('n dropped features = ', len(to_drop))
print('Number of Training features after Exclusion of highly correlated features =', X_train.shape[1])
X_test = drop_correlated_features_evaluation(X_test, to_drop)

n dropped features =  3264
Number of Training features after Exclusion of highly correlated features = 1223


In [7]:
X_train.shape

(471, 1223)

In [8]:
X_test.shape

(84, 1223)

In [9]:
y_train.shape

(471,)

In [10]:
y_test.shape

(84,)

## A Random Survival Forest model is fit on the remaining candidate variables and a ranked list of variable importance scores is obtainedby ten permutations of random shuffling.

In [11]:
# basic rsf for feature selection
basic_rsf = RandomSurvivalForest(n_estimators=1000,
                               min_samples_split=10,
                               min_samples_leaf=15,
                               max_features="sqrt",
                               n_jobs=-1,
                               random_state=0)

basic_rsf.fit(X_train, y_train)

print('BASIC RSF TRAINING c-Index', basic_rsf.score(X_train, y_train))
print('BASIC RSF TESTING  c-Index', basic_rsf.score(X_test, y_test))

BASIC RSF TRAINING c-Index 0.878042189263516
BASIC RSF TESTING  c-Index 0.7456318569687119


In [ ]:
perm = PermutationImportance(basic_rsf, n_iter=10, random_state=0)

train_feature_importance = perm.fit(X_train, y_train)
feature_names = X_train.columns.tolist()

## The final survival model is fit on the 30 top most important variables.

In [ ]:
important_features_30 = eli5.explain_weights_df(perm, feature_names=feature_names).iloc[:30]['feature'].values

    
important_30_rsf = RandomSurvivalForest(n_estimators=1000,
                                   min_samples_split=10,
                                   min_samples_leaf=15,
                                   max_features="sqrt",
                                   n_jobs=-1,
                                   random_state=0)

   
important_30_rsf.fit(X_train[important_features_30], y_train)

In [19]:
print('RSF-30 TRAINING c-Index', important_30_rsf.score(X_train[important_features_30], y_train))
print('RSF-30 TESTING  c-Index', important_30_rsf.score(X_test[important_features_30], y_test))

RSF-30 TRAINING c-Index 0.8503346209483841
RSF-30 TESTING  c-Index 0.8232425843153189


## Prediction of risk scores.

In [ ]:
df_train['SurvPred'] = important_30_rsf.predict(df_train[important_features_30])
df_test['SurvPred']  = important_30_rsf.predict(df_test[important_features_30])

In [ ]:
df_train['DATA_SPLIT'] = 'Train'
df_test['DATA_SPLIT'] = 'Test'

df_master = pd.concat([df_train, df_test])
df_master.to_csv('PerformanceEvaluation.csv')

## To gain further insight and interpretability of the final model, we obtain variable importance scores by ten permutations of random shuffling for all 30 included variables. 

In [43]:
perm_30_rsf = PermutationImportance(important_30_rsf, n_iter=10, random_state=0)

train_feature_importance = perm_30_rsf.fit(X_train[important_features_30], y_train)
feature_names_30_rsf = X_train[important_features_30].columns.tolist()


eli5.explain_weights_df(perm_30_rsf, feature_names=feature_names_30_rsf).iloc[:30].to_csv('FeatureImportances.csv')